### ACS Poverty Status by Race/Ethnicity in Phoenix Urban Villages, City of Phoenix, U.S., Maricopa County, Arizona, and Comparable Cities

For population by age 1-year data
- https://www.census.gov/data/developers/data-sets/acs-1year.html
- https://api.census.gov/data/2010/acs/acs1/variables.html
- https://api.census.gov/data/2021/acs/acs1/variables.html

For population by age 5-year data
- https://www.census.gov/data/developers/data-sets/acs-5year.html
- https://api.census.gov/data/2013/acs/acs5/variables.html
- https://api.census.gov/data/2021/acs/acs5/variables.html

** Note: Census Block Groups (which are aggregated to Phoenix Urban Village areas) are not available until 2013. 

In [1]:
import pandas as pd
import math
import numpy as np

In [2]:
import getters as get
import utilcalcs as calc
import geo_agg
from acs_poverty_vars import *

In [3]:
#Search parameters
y1 = '2021'
y0_5 = '2013' #for urban villages
y0_1 = '2010'

source5 = 'acs/acs5'
source1 = 'acs/acs1'

### Calculate poverty by race/ethnicity for multiple geos & source years

# TEST THIS FUNCTION//CHECK APPLICATION ACROSS YR1 YR5, URBAN VILLAGE/OTHER GEOS!!

In [4]:
#def clean_table(df):   
#    df = df.filter(regex='(?<!A)$',axis=1) #drop non-estimate columns
#    df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']] #move id to first col
#    df = clean_data(df,df.columns)
#    return df

def calc_pov(df):
    #df = df[['GEO_ID']+[col for col in df.columns if col != 'GEO_ID']]
    df = get.clean_table(df,df.columns)
    df.rename(columns=pov_rename,inplace=True)
    df['oth_E']=df.loc[:,oth_e].sum(axis=1)
    df['oth_M']=df.apply(lambda x: (calc.get_moe(x[oth_m])),axis=1)
    df['oth_pov_E']=df.loc[:,oth_pov_e].sum(axis=1)
    df['oth_pov_M']=df.apply(lambda x: (calc.get_moe(x[oth_pov_m])),axis=1)
    df = df.drop(oth,axis=1)
    return df

## Population in poverty by race/ethnicity - larger geos
### Note: 2010 to 2021 ACS 1YR comparison

In [5]:
def make_big_geos(source,year,cols):
    ph = get.get_phx(source,year,cols)
    mar = get.get_maricopa(source,year,cols)
    us = get.get_us(source,year,cols)
    az = get.get_az(source,year,cols)
    cit = get.get_comp_cities(source,year,cols)
    Y = pd.concat([ph,mar,us,az,cit])
    Y = Y.drop(['NAME','us','state'],axis=1)
    Y = calc_pov(Y)
    Y = geo_agg.sumgeo_cv(Y,'GEO_ID')
    
    for col in Y.columns[1:]:
        Y.rename(columns={col:f'{col[:-1]}{year[-2:]}{col[-1:]}'},inplace=True)
    return Y

In [ ]:
Y1 = make_big_geos(source1,y1,col_pov)
Y0 = make_big_geos(source1,y0_1,col_pov)

In [ ]:
rename_geos = {'0455000':'Phoenix', '04013':'Maricopa', '0100000US':'US', '0400000US04':'AZ',\
               '0473000':'Tempe','0465000':'Scottsdale', '0427820':'Glendale',\
               '4865000':'San Antonio', '4819000':'Dallas', '1235000':'Jacksonville'}

In [ ]:
for df in [Y0,Y1]:
    df['GEO_ID'] = df.GEO_ID.map(rename_geos)

In [ ]:
Y1.head(3)

In [ ]:
Y0.head(3)

with pd.ExcelWriter(f'output/poverty.xlsx') as writer:
    Y1.to_excel(writer, sheet_name="2021_1YR", index=False)
    Y0.to_excel(writer, sheet_name="2010_1YR", index=False)

# CBGS DO NOT HAVE INFORMATION ABOUT POPULATION LIVING IN POVERTY - REVISE // USE PUMA BOUNDARIES FOR CONSISTENCY OR EXPLORE TRACT//URBAN VILLAGE APPROXIMATION OPPORTUNITY

### Urban Villages, City, U.S. with ACS 5-year
#### Note: base year is 2013 5YR for consistency

In [6]:
def make_uv_pov(df,year): 
    df = calc_pov(df)
    df = geo_agg.make_uv(df,int(year))

    for col in df.columns[1:]:
        df.rename(columns={col:f'{col[:-1]}{year[-2:]}{col[-1:]}'},inplace=True)
    return df

In [7]:
bg_13 = get.get_bgp(source5,y0_5,col_pov)
bg_20 = get.get_bgp(source5,y1,col_pov)

In [9]:
bg_13.head()

,GEO_ID,B17001_001E,B17001_001M,B17001_002E,B17001_002M,B17001A_001E,B17001A_001M,B17001A_002E,B17001A_002M,B17001B_001E,...,B17001G_002E,B17001G_002M,B17001H_001E,B17001H_001M,B17001H_002E,B17001H_002M,B17001I_001E,B17001I_001M,B17001I_002E,B17001I_002M
0,040130931052,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,040130931051,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,040130930022,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,040130930021,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,040130930023,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [11]:
uv_13 = make_uv_pov(bg_13,y0_5)
uv_20 = make_uv_pov(bg_20,y0_5)

In [14]:
uv_20

,name,whi_13E,whi_13M,whi_13C,whnon_pov_13E,whnon_pov_13M,whnon_pov_13C,bla_pov_13E,bla_pov_13M,bla_pov_13C,...,whi_pov_13C,oth_13E,oth_13M,oth_13C,his_13E,his_13M,his_13C,bla_13E,bla_13M,bla_13C
0,Laveen,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
1,Deer Valley,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
2,Maryvale,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
3,North Mountain,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
4,Paradise Valley,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
5,Encanto,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
6,Camelback East,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
7,South Mountain,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
8,Alhambra,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0
9,Ahwatukee Foothills,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0,...,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0,0


##### Large geographies

In [ ]:
Y1 = make_big_geos(source5,y1,col_pov)
Y0 = make_big_geos(source5,y0_5,col_pov)
for df in [Y0,Y1]:
    df['GEO_ID'] = df.GEO_ID.map(rename_geos)